# Impor library

In [1]:
!pip install gdown

In [8]:
import gdown
import zipfile
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import torch
from datasets import Dataset
from transformers import pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
device = 0 if torch.cuda.is_available() else -1

# Download model yang sudah di fine-tune

In [3]:
file_id = "1a1u4PVrkBD7RdMwuJaoUy_2sJzjMCbuY"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "indobert.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1a1u4PVrkBD7RdMwuJaoUy_2sJzjMCbuY
From (redirected): https://drive.google.com/uc?id=1a1u4PVrkBD7RdMwuJaoUy_2sJzjMCbuY&confirm=t&uuid=6ed15454-f416-4dce-a173-78997c82675a
To: /content/indobert.zip
100%|██████████| 1.25G/1.25G [00:08<00:00, 153MB/s]


'indobert.zip'

# Ekstrak model

In [4]:
with zipfile.ZipFile("indobert.zip", 'r') as zip_ref:
    zip_ref.extractall("indobert-finetuned-youtube")

# panggil data yang sudah dilabeli sebagian

In [5]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRqrNZCwGD4oXSMsdkiB3c5V5DFw3LTwF7grGwpgYpefWlfMgcncNZcWqbApGwj_GYdm92-aAYKsTWQ/pub?output=csv")

# Proses pelabelan model

In [10]:
df_ori=df.copy()

In [11]:
# -------------------------------------------------
# 1.  Make sure the column is really called "text"
# -------------------------------------------------
df_pred = df.rename(columns={'text': 'ori','cleaned_nostop': 'text'})     # <- change once
texts    = df_pred['text'].astype(str).tolist()              # plain list

# -------------------------------------------------
# 2.  Load the fine-tuned IndoBERT classifier
# -------------------------------------------------
device_id = 0 if torch.cuda.is_available() else -1
clf = pipeline(
    task="sentiment-analysis",
    model="indobert-finetuned-youtube",
    tokenizer="indobert-finetuned-youtube",
    device=device_id
)

# -------------------------------------------------
# 3.  Batch inference with safe truncation
# -------------------------------------------------
preds = clf(
    texts,                 # list of raw strings
    batch_size=32,         # adjust for your GPU
    truncation=True,       # <- stops length crash
    max_length=512,
    padding=True
)

# -------------------------------------------------
# 4.  Attach predictions to the original DataFrame
# -------------------------------------------------
df['fine_tuned_label'] = [p['label'] for p in preds]
# (Optionally) df['fine_tuned_score'] = [p['score'] for p in preds]

print(df[['text', 'fine_tuned_label']].head())

Device set to use cuda:0


                                                text fine_tuned_label
0  Malakan, menurut kalian Danantara lebih banyak...          LABEL_0
1                                 Mumet kondisi skrg          LABEL_2
2  analogi pernikahan dengan papa dana itu agak k...          LABEL_1
3  Dalam waktu kedepan tak lama lg, akan muncul b...          LABEL_0
4                           Opportunity apa ya bang?          LABEL_1


In [15]:
pd.set_option('display.max_colwidth', None)
df.sample(10,random_state=20)

,Unnamed: 0,text,manual label,cleaned_nostop,fine_tuned_label
33141,33468,"mantap pak! korupsi ok pak, tapi pak... jangan di korupsi nya keterlaluan ya... di sisain dikit gitu...",NaN,mantap korupsi ok korupsi nya ya sisain dikit gitu,LABEL_2
2662,2682,Jangan sampai dana jadi kayak eka di home sweet loan 😂,NaN,dana kayak eka home sweet loan,LABEL_1
27138,27401,"DANANTARA ITU CELENGAN UANG. NAMANYA CELENGAN BISA LUMUTAN, TERBAKAR, HILANG DAN DIREKAYASA.\nJika orang2 DANA ANTARA TIDAK LAGI IKUT BERKUASA PERIODE YANG AKAN DATANG, tentu MEREKA AKAN TETAP BERKUASA PADA DANANTARA. JIKA MEREKA DITUNTUT RAKYAT, MEREKA BISA KABUR KE NEGARA ASING TOH?\nInga\ninga\nInga\nKABUR AJA DULu.",NaN,danantara celengan uang namanya celengan lumutan terbakar hilang direkayasa orang dana berkuasa periode berkuasa danantara dituntut rakyat kabur negara asing inga inga inga kabur aja,LABEL_1
37056,37400,"Ini lg wak min,udh udh lanjut usia masih doyan mau buat onar di publik.banyak beramal jaga kesehatan jaga mulut berbanyak doa ny.negara ini ada presiden ny.nggk perlu wak min ikutan mau ngurusi.jaman wakmin,wak min cm bisa jadi penjilat negara sampai negara lebih hancur gara gara ulah mulut bpk.",NaN,lg wak minudh udh usia doyan onar publikbanyak beramal jaga kesehatan jaga mulut berbanyak doa nynegara presiden nynggk wak min ikutan ngurusijaman wakminwak min cm penjilat negara negara hancur gara gara ulah mulut bpk,LABEL_1
12058,12188,"Saham smgr, pgas, kontruksi....hancur lebur",NaN,saham smgr pgas kontruksihancur lebur,LABEL_2
38034,38386,tumben bener ni orang,NaN,tumben bener ni orang,LABEL_2
31600,31909,Lumayan menambah wawasan sy dr podcast seru ini. Makasih om pandu,NaN,lumayan menambah wawasan sy dr podcast seru makasih om pandu,LABEL_1
16696,16861,Bkn cmn taksin sanawatra ada mantan PM Australia juga😂,NaN,bkn cmn taksin sanawatra mantan pm australia,LABEL_1
18987,19177,ayo Desak DPR untuk sahkan UU perampasan aset pada koruptor,NaN,ayo desak dpr sahkan uu perampasan aset koruptor,LABEL_1
1121,1133,Bang katanya danantara ga transparan dan kebal hukum? Beneran ga bang,NaN,bang danantara ga transparan kebal hukum beneran ga bang,LABEL_2


# Simpan data

In [17]:
df.to_csv('data_bersih_klasifikasi.csv')